# Import Libraries 

In [312]:
import pandas as pd
import numpy as np
from tabulate import tabulate
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler


# 1. Global Variables

In [308]:
global_vars = {
    "DATASET_FILE_PREFIX" : "C:\\Users\\sapta\\Documents\\git\\DeepLearning\\QuickML_API\\_docs\\dataset\\",

    "split_profile" : {
        'independent_vars' : ['Country', 'Age', 'Salary'],
        'dependent_vars' : ['Purchased'],
        'train_size' : 0.7
    },

    "impute_strategy" : {
            'missing_value' : np.nan,
            'strategy': 'mean'
    },

    "categorical_cols" : {
        'independent' : ['Country'],
        'dependent' : ['Purchased']
    }
}

In [135]:
def print_tab(df : pd.DataFrame, head = False) -> None:
    """prints pandas dataframe in tabular form 

    Args:
        df (pd.DataFrame): dataframe to print
        head (bool): if set to False (default) prints whole table
    """

    if head:
        print(tabulate(df.head(), headers = 'keys', tablefmt = 'pretty'))
    else:
        print(tabulate(df, headers = 'keys', tablefmt = 'pretty'))

# 2. Loading Dataset

In [136]:
def load_dataset(dataset_filename):
    DATASET_FILE_PREFIX = global_vars['DATASET_FILE_PREFIX']
    source_dataframe = pd.read_csv(DATASET_FILE_PREFIX + dataset_filename)
    return source_dataframe

In [137]:
source_dataframe = load_dataset(dataset_filename="data_preprocessing.csv")
print_tab(source_dataframe)

+---+---------+------+---------+-----------+
|   | Country | Age  | Salary  | Purchased |
+---+---------+------+---------+-----------+
| 0 | France  | 44.0 | 72000.0 |    No     |
| 1 |  Spain  | 27.0 | 48000.0 |    Yes    |
| 2 | Germany | 30.0 | 54000.0 |    No     |
| 3 |  Spain  | 38.0 | 61000.0 |    No     |
| 4 | Germany | 40.0 |   nan   |    Yes    |
| 5 | France  | 35.0 | 58000.0 |    Yes    |
| 6 |  Spain  | nan  | 52000.0 |    No     |
| 7 | France  | 48.0 | 79000.0 |    Yes    |
| 8 | Germany | 50.0 | 83000.0 |    No     |
| 9 | France  | 37.0 | 67000.0 |    Yes    |
+---+---------+------+---------+-----------+


# 3. Dependent - Independent variable Split

In [226]:
def get_DI_split(source_dataframe : pd.DataFrame, conv_to_np = False) -> dict:
    
    split_profile = global_vars['split_profile']
    X = source_dataframe[ split_profile['independent_vars'] ]
    y = source_dataframe[ split_profile['dependent_vars'] ]

    if conv_to_np:
        X = X.to_numpy()
        y = y.to_numpy()
    
    return {'X':X , 'y':y}

In [263]:
split_profile = global_vars['split_profile']

splitted_data = get_DI_split(source_dataframe=source_dataframe)
print('Independent (X)')
print_tab(splitted_data["X"])

print('Dependent (y)')
print_tab(splitted_data["y"])

Independent (X)
+---+---------+------+---------+
|   | Country | Age  | Salary  |
+---+---------+------+---------+
| 0 | France  | 44.0 | 72000.0 |
| 1 |  Spain  | 27.0 | 48000.0 |
| 2 | Germany | 30.0 | 54000.0 |
| 3 |  Spain  | 38.0 | 61000.0 |
| 4 | Germany | 40.0 |   nan   |
| 5 | France  | 35.0 | 58000.0 |
| 6 |  Spain  | nan  | 52000.0 |
| 7 | France  | 48.0 | 79000.0 |
| 8 | Germany | 50.0 | 83000.0 |
| 9 | France  | 37.0 | 67000.0 |
+---+---------+------+---------+
Dependent (y)
+---+-----------+
|   | Purchased |
+---+-----------+
| 0 |    No     |
| 1 |    Yes    |
| 2 |    No     |
| 3 |    No     |
| 4 |    Yes    |
| 5 |    Yes    |
| 6 |    No     |
| 7 |    Yes    |
| 8 |    No     |
| 9 |    Yes    |
+---+-----------+


# 4. Remove Missing Data

In [264]:
def get_col_list_with_null(dataframe : pd.DataFrame) -> list:

    col_list_with_null = [] 
    for column in dataframe.columns:
        if dataframe[column].isnull().any():
            col_list_with_null.append(column)
    return col_list_with_null

In [265]:
col = get_col_list_with_null(dataframe=splitted_data["X"])
print(col)

['Age', 'Salary']


In [266]:
def remove_missing_data(dataframe : pd.DataFrame ) -> pd.DataFrame:
    
    impute_strategy = global_vars['impute_strategy']

    # instantiate imputer object
    si = SimpleImputer(missing_values=impute_strategy['missing_value'], 
                       strategy=impute_strategy['strategy'])

    # get all column names with missing data
    col_list_with_null = get_col_list_with_null(dataframe=dataframe)

    # replace all missing values 
    df_copy = dataframe.copy()
    df_copy[col_list_with_null] = si.fit(df_copy[col_list_with_null]).transform(df_copy[col_list_with_null])

    return df_copy

In [267]:
# get splitted DF
split_df = get_DI_split(source_dataframe=source_dataframe)

# get imputed DF
splitted_data["X"] = remove_missing_data(dataframe=splitted_data["X"])

# print imputed
print_tab(splitted_data["X"])

+---+---------+-------------------+-------------------+
|   | Country |        Age        |      Salary       |
+---+---------+-------------------+-------------------+
| 0 | France  |       44.0        |      72000.0      |
| 1 |  Spain  |       27.0        |      48000.0      |
| 2 | Germany |       30.0        |      54000.0      |
| 3 |  Spain  |       38.0        |      61000.0      |
| 4 | Germany |       40.0        | 63777.77777777778 |
| 5 | France  |       35.0        |      58000.0      |
| 6 |  Spain  | 38.77777777777778 |      52000.0      |
| 7 | France  |       48.0        |      79000.0      |
| 8 | Germany |       50.0        |      83000.0      |
| 9 | France  |       37.0        |      67000.0      |
+---+---------+-------------------+-------------------+


# 5. Encoding of Categorical Variables 


## 5.1. Individual implementaion of encoders

### 5.1.1. Loding dataframe

In [297]:
dataframe_X = splitted_data["X"]
dataframe_y = splitted_data["y"]

catagorical_cols_X = global_vars["categorical_cols"]["independent"]
catagorical_cols_y = global_vars["categorical_cols"]["dependent"]

print(f'Categorical  Columns = {catagorical_cols_X}')
print_tab(dataframe_X)

print(f'Categorical  Columns = {catagorical_cols_y}')
print_tab(dataframe_y)

Categorical  Columns = ['Country']
+---+---------+-------------------+-------------------+
|   | Country |        Age        |      Salary       |
+---+---------+-------------------+-------------------+
| 0 | France  |       44.0        |      72000.0      |
| 1 |  Spain  |       27.0        |      48000.0      |
| 2 | Germany |       30.0        |      54000.0      |
| 3 |  Spain  |       38.0        |      61000.0      |
| 4 | Germany |       40.0        | 63777.77777777778 |
| 5 | France  |       35.0        |      58000.0      |
| 6 |  Spain  | 38.77777777777778 |      52000.0      |
| 7 | France  |       48.0        |      79000.0      |
| 8 | Germany |       50.0        |      83000.0      |
| 9 | France  |       37.0        |      67000.0      |
+---+---------+-------------------+-------------------+
Categorical  Columns = ['Purchased']
+---+-----------+
|   | Purchased |
+---+-----------+
| 0 |    No     |
| 1 |    Yes    |
| 2 |    No     |
| 3 |    No     |
| 4 |    Yes    |


### 5.1.2. Label Encoding

In [296]:
dataframe_X = splitted_data["X"]

label_encoder = LabelEncoder()

dataframe_X = splitted_data["X"].copy()
dataframe_y = splitted_data["y"].copy()

for column in catagorical_cols_X:
    dataframe_X[column] = label_encoder.fit_transform(dataframe_X[column])

for column in catagorical_cols_y:
    dataframe_y[column] = label_encoder.fit_transform(dataframe_y[column])

print("X after encoding")
print_tab(dataframe_X)

print("y after encoding")
print_tab(dataframe_y)

X after encoding
+---+---------+-------------------+-------------------+
|   | Country |        Age        |      Salary       |
+---+---------+-------------------+-------------------+
| 0 |   0.0   |       44.0        |      72000.0      |
| 1 |   2.0   |       27.0        |      48000.0      |
| 2 |   1.0   |       30.0        |      54000.0      |
| 3 |   2.0   |       38.0        |      61000.0      |
| 4 |   1.0   |       40.0        | 63777.77777777778 |
| 5 |   0.0   |       35.0        |      58000.0      |
| 6 |   2.0   | 38.77777777777778 |      52000.0      |
| 7 |   0.0   |       48.0        |      79000.0      |
| 8 |   1.0   |       50.0        |      83000.0      |
| 9 |   0.0   |       37.0        |      67000.0      |
+---+---------+-------------------+-------------------+
y after encoding
+---+-----------+
|   | Purchased |
+---+-----------+
| 0 |     0     |
| 1 |     1     |
| 2 |     0     |
| 3 |     0     |
| 4 |     1     |
| 5 |     1     |
| 6 |     0     |
| 

### 5.1.3. One Hot Encoding (OHE)

In [293]:
dataframe_X = splitted_data["X"].copy()
for column in catagorical_cols_X:
    oh_encoder = OneHotEncoder(handle_unknown='ignore')
    oh_df = pd.DataFrame(oh_encoder.fit_transform(dataframe_X[[column]]).toarray())
    dataframe_X = dataframe_X.join(oh_df)
    
print_tab(dataframe_X)

+---+---------+-------------------+-------------------+-----+-----+-----+
|   | Country |        Age        |      Salary       |  0  |  1  |  2  |
+---+---------+-------------------+-------------------+-----+-----+-----+
| 0 | France  |       44.0        |      72000.0      | 1.0 | 0.0 | 0.0 |
| 1 |  Spain  |       27.0        |      48000.0      | 0.0 | 0.0 | 1.0 |
| 2 | Germany |       30.0        |      54000.0      | 0.0 | 1.0 | 0.0 |
| 3 |  Spain  |       38.0        |      61000.0      | 0.0 | 0.0 | 1.0 |
| 4 | Germany |       40.0        | 63777.77777777778 | 0.0 | 1.0 | 0.0 |
| 5 | France  |       35.0        |      58000.0      | 1.0 | 0.0 | 0.0 |
| 6 |  Spain  | 38.77777777777778 |      52000.0      | 0.0 | 0.0 | 1.0 |
| 7 | France  |       48.0        |      79000.0      | 1.0 | 0.0 | 0.0 |
| 8 | Germany |       50.0        |      83000.0      | 0.0 | 1.0 | 0.0 |
| 9 | France  |       37.0        |      67000.0      | 1.0 | 0.0 | 0.0 |
+---+---------+-------------------+---

### 5.1.4. OHE with Dummy variables 

In [294]:
dataframe_X = splitted_data["X"].copy()
for column in catagorical_cols_X:
    # one-hot encoding for each categical var
    dummy_df = pd.get_dummies(dataframe_X[column], columns=[column], prefix=column)
    print_tab(dummy_df)  # dummy var table for each encoding 
    dataframe_X = dataframe_X.join(dummy_df).drop(column, axis = 1) # upate dataframe by joining 
    
print_tab(dataframe_X)

+---+----------------+-----------------+---------------+
|   | Country_France | Country_Germany | Country_Spain |
+---+----------------+-----------------+---------------+
| 0 |       1        |        0        |       0       |
| 1 |       0        |        0        |       1       |
| 2 |       0        |        1        |       0       |
| 3 |       0        |        0        |       1       |
| 4 |       0        |        1        |       0       |
| 5 |       1        |        0        |       0       |
| 6 |       0        |        0        |       1       |
| 7 |       1        |        0        |       0       |
| 8 |       0        |        1        |       0       |
| 9 |       1        |        0        |       0       |
+---+----------------+-----------------+---------------+
+---+-------------------+-------------------+----------------+-----------------+---------------+
|   |        Age        |      Salary       | Country_France | Country_Germany | Country_Spain |
+---+---

## 5.2. Categotical Encoding with OHE 

In [300]:
def  cat_endcoing(dataframe : pd.DataFrame, categorical_cols : list, ohe = True):
    """_summary_

    Args:
        dataframe_X (pd.DataFrame): _description_
        categorical_cols (list): _description_
        ohe (bool): _description__

    Returns:
        _type_: _description_
    """
    df = dataframe.copy()
    if ohe:  # one hot encoding 
        for column in categorical_cols:
            dummy_df = pd.get_dummies(df[column], columns=[column], prefix=column)  # dummy var table for each encoding 
            df = df.join(dummy_df).drop(column, axis = 1) # upate dataframe by joining 

    else:    # label encoding
        label_encoder = LabelEncoder() 
        for column in categorical_cols:
            df[column] = label_encoder.fit_transform(df[column])
    
    return df

In [301]:
dataframe_X = cat_endcoing(dataframe=dataframe_X, categorical_cols=global_vars['categorical_cols']['independent'], ohe=True)
dataframe_y = cat_endcoing(dataframe=dataframe_y, categorical_cols=global_vars['categorical_cols']['dependent'], ohe=False)

print_tab(dataframe_X)
print_tab(dataframe_y)

+---+-------------------+-------------------+----------------+-----------------+---------------+
|   |        Age        |      Salary       | Country_France | Country_Germany | Country_Spain |
+---+-------------------+-------------------+----------------+-----------------+---------------+
| 0 |       44.0        |      72000.0      |      1.0       |       0.0       |      0.0      |
| 1 |       27.0        |      48000.0      |      0.0       |       0.0       |      1.0      |
| 2 |       30.0        |      54000.0      |      0.0       |       1.0       |      0.0      |
| 3 |       38.0        |      61000.0      |      0.0       |       0.0       |      1.0      |
| 4 |       40.0        | 63777.77777777778 |      0.0       |       1.0       |      0.0      |
| 5 |       35.0        |      58000.0      |      1.0       |       0.0       |      0.0      |
| 6 | 38.77777777777778 |      52000.0      |      0.0       |       0.0       |      1.0      |
| 7 |       48.0        |     

# 6. Train Test Split

In [315]:
def tt_split(dataframe_X : pd.DataFrame, dataframe_y : pd.DataFrame) -> dict :
    """performs train-test-split on the given dataset

    Args:
        dataframe (pd.DataFrame): source dataframe 
        split_profile (dict): dictionary must cotain a key 'tt_ratio' (float) [0.0 - 1.0] denotes training proportion 

    Returns:
        dict: _description_
    """
    X_train, X_test, y_train, y_test = train_test_split(dataframe_X, dataframe_y, train_size=global_vars['split_profile']['train_size'])
    return {
        'X_train' : X_train,
        'X_test' : X_test,
        'y_train' : y_train,
        'y_test' : y_test
    }

In [319]:
result = tt_split(dataframe_X=dataframe_X, dataframe_y=dataframe_y)

for key in result.keys():
    print(key)
    print_tab(result[key])

X_train
+---+-------------------+-------------------+----------------+-----------------+---------------+
|   |        Age        |      Salary       | Country_France | Country_Germany | Country_Spain |
+---+-------------------+-------------------+----------------+-----------------+---------------+
| 0 |       44.0        |      72000.0      |      1.0       |       0.0       |      0.0      |
| 9 |       37.0        |      67000.0      |      1.0       |       0.0       |      0.0      |
| 5 |       35.0        |      58000.0      |      1.0       |       0.0       |      0.0      |
| 8 |       50.0        |      83000.0      |      0.0       |       1.0       |      0.0      |
| 7 |       48.0        |      79000.0      |      1.0       |       0.0       |      0.0      |
| 4 |       40.0        | 63777.77777777778 |      0.0       |       1.0       |      0.0      |
| 6 | 38.77777777777778 |      52000.0      |      0.0       |       0.0       |      1.0      |
+---+-----------------

# Feature Scalling 


In [329]:
def feature_scalling(tt_splitted_dfs : dict , scale_dummies = True) ->dict:
  """_summary_

  Args:
      tt_splitted_dfs (dict): _description_
      scale_dummies (bool, optional): _description_. Defaults to True.

  Returns:
      dict: _description_
  """
    
  sc_X = StandardScaler()
  if not scale_dummies:
    # non-categoriccal independent vars
    scalable_features = list(set(global_vars['split_profile']['independent_vars']) - set(global_vars['categorical_cols']['independent']))

  return{
    'X_train' : sc_X.fit_transform(tt_splitted_dfs['X_train']),
    'X_test' : sc_X.transform(tt_splitted_dfs['X_test']),
    'y_train' : tt_splitted_dfs['y_train'],
    'y_test' : tt_splitted_dfs['y_test']
  }

In [331]:
scalled_dfs = feature_scalling(tt_splitted_dfs=result, scale_dummies=False)

for key in result.keys():
    print(key)
    print_tab(scalled_dfs[key])

X_train
+----------------------+----------------------+---------------------+---------------------+--------------------+
|          0           |          1           |          2          |          3          |         4          |
+----------------------+----------------------+---------------------+---------------------+--------------------+
|  0.4150371955802574  |  0.4069530487552628  | 0.8660254037844387  | -0.6324555320336759 | -0.408248290463863 |
| -0.9209584485868486  | -0.08046219975389231 | 0.8660254037844387  | -0.6324555320336759 | -0.408248290463863 |
| -1.3026714897774505  | -0.9578096470703715  | 0.8660254037844387  | -0.6324555320336759 | -0.408248290463863 |
|  1.5601763191520626  |  1.479266595475404   | -1.1547005383792517 |  1.58113883008419   | -0.408248290463863 |
|  1.1784632779614608  |   1.08933439666808   | 0.8660254037844387  | -0.6324555320336759 | -0.408248290463863 |
| -0.34838888680094604 | -0.3945742487931253  | -1.1547005383792517 |  1.58113883008419 

In [323]:
A = [1,2,3]
B = [2]

list(set(A) - set(B))

[1, 3]